In [ ]:
!pip install opencv-python Pillow scikit-image

In [ ]:
!curl -O "https://i.redd.it/1sme6te6knf01.jpg" 

Make a folder 'images' and move the image there, renaming it to car.jpg

In [ ]:
mkdir images && mv 1sme6te6knf01.jpg images/car.jpg

In [ ]:
import matplotlib.pyplot as plt

img_path = 'images/car.jpg'

In [ ]:
# Load with open-cv
import cv2
# open-cv represents RGB images as BGR (reverse order)
img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

plt.imshow(img)
plt.show()

In [ ]:
# Load with scikit
from skimage import io

img = io.imread(img_path)

plt.imshow(img)
plt.show()

In [ ]:
# Load with PIL
from PIL import Image

img =  Image.open(img_path)

plt.imshow(img)
plt.show()

When reading the image as a NumPy array `ndarray`, various image processing can be performed using NumPy functions.  

By using an ndarray, you can get and set (change) pixel values, trim images, concatenate images, etc. Those who are familiar with NumPy can do various image processing without using libraries such as OpenCV.  

Even when using OpenCV, OpenCV for Python treats image data as ndarray, so it is useful to know how to use NumPy (ndarray). In addition to OpenCV, many libraries such as scikit-image treat images as ndarray.

#### Manipulating images with numpy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

image = io.imread(img_path)

# set rows 0 - 100 to zero
image[:100] = 0

# create a mask from all pixels that are 'darker' than 69
mask = image < 69
# color the mask white
image[mask] = 255

plt.figure(figsize=(6, 6))
plt.imshow(image, cmap='gray')
plt.axis('off')
plt.show()

### basic image processing with skimage

### threshold

In [ ]:
from skimage import data
from skimage import filters
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
  
image = io.imread(img_path)
# conversion to grayscale is neccessary for most skimage functions
grayscale = rgb2gray(image)
  
# Setting the plot size
plt.figure(figsize=(16, 8))
  
for i in range(10):
    
    # Iterating different thresholds
    binarized_gray = (grayscale > i*0.1)*1
    plt.subplot(2,5,i+1)
    
    # Rounding of the threshold
    # value to 1 decimal point
    plt.title("Threshold: >"+str(round(i*0.1,1)))

    # Displaying the binarized image
    # of various thresholds
    plt.imshow(binarized_gray, cmap = 'gray')
    
plt.tight_layout()

#### Edge Detection

In [ ]:
import numpy as np

from skimage.color import rgb2gray
from skimage import filters
# from skimage.util import compare_images

#load image and convert to grayscale for edge detection
image = io.imread(img_path)
grayscale = rgb2gray(image)

# there are multiple filters to choose from
edge_roberts = filters.roberts(grayscale)
edge_sobel = filters.sobel(grayscale)

# set up the plot for comparison
fig, axes = plt.subplots(ncols=2, sharex=True, sharey=True,
                         figsize=(16, 16))

axes[0].imshow(edge_roberts, cmap=plt.cm.gray)
axes[0].set_title('Roberts Edge Detection')

axes[1].imshow(edge_sobel, cmap=plt.cm.gray)
axes[1].set_title('Sobel Edge Detection')

# disable axis
for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
from skimage.exposure import rescale_intensity

image = io.imread(img_path)
grayscale = rgb2gray(image)

# invert the edges
image = rescale_intensity(1 - filters.sobel(grayscale))

plt.figure(figsize=(8, 8))
plt.imshow(image, cmap='gray')
plt.axis('off')
plt.show()

#### entropy (a measure of how "random" a probability distribution is)

In [ ]:
from skimage import filters, morphology, color

entropy_width = 20

image = io.imread(img_path)
grayscale = rgb2gray(image)

# calculate entropy
entropy = filters.rank.entropy(grayscale, morphology.disk(entropy_width))

# plot it
fig, ax = plt.subplots(figsize=(8,8))

ax.axis('off')
ax.imshow(entropy)

#### Feature Detection

In [ ]:
from skimage.feature import CENSURE

image = io.imread(img_path)
grayscale = rgb2gray(image)

detector = CENSURE()

fig, ax = plt.subplots(figsize=(8, 8))

detector.detect(grayscale)

ax.imshow(image, alpha=0.5)
ax.scatter(detector.keypoints[:, 1], detector.keypoints[:, 0],
              2 ** detector.scales, facecolors='none', edgecolors='r')
ax.axis('off')

plt.tight_layout()
plt.show()

#### Segmentation

In [ ]:
# Importing required boundaries
from skimage.segmentation import slic, mark_boundaries

# Setting the plot figure
plt.figure(figsize=(12, 12))

image = io.imread(img_path)
grayscale = rgb2gray(image)

# Applying SLIC segmentation
# for the edges to be drawn over
img_segments = slic(image, n_segments=50, compactness=1)

plt.subplot(1, 2, 1)

# Plotting the original image
plt.imshow(image)

# Detecting boundaries for labels
plt.subplot(1, 2, 2)

# Plotting the ouput of marked_boundaries
# function i.e. the image with segmented boundaries
plt.imshow(mark_boundaries(image, img_segments))

In [ ]:
from skimage.color import label2rgb

plt.figure(figsize=(12, 12))

plt.imshow(label2rgb(img_segments, image, kind = 'avg'))

Semantic Segmentation:

In [ ]:
from IPython.display import Video
Video(url='https://blogs.nvidia.com/wp-content/uploads/2019/10/Panoptic-Segmentation_Video-Trimmed.mp4', width=600)

#### Haar Cascade

In [ ]:
!curl https://thispersondoesnotexist.com/image --output images/person.png

In [ ]:
from skimage import data
from skimage.feature import Cascade
from skimage import io
from skimage.color import rgb2gray


import matplotlib.pyplot as plt
from matplotlib import patches

# Load the trained file from the module root.
trained_file = data.lbp_frontal_face_cascade_filename()

# Initialize the detector cascade.
detector = Cascade(trained_file)

image = io.imread("images/person.png")

detected = detector.detect_multi_scale(img=image,
                                       scale_factor=1.2,
                                       step_ratio=1,
                                       min_size=(60, 60),
                                       max_size=(1024, 1024))

# plt.figure(figsize=(8, 8))
plt.imshow(image)
img_desc = plt.gca()
plt.set_cmap('gray')

for patch in detected:

    img_desc.add_patch(
        patches.Rectangle(
            (patch['c'], patch['r']),
            patch['width'],
            patch['height'],
            fill=False,
            color='r',
            linewidth=2
        )
    )


# ax.axis('off')
# plt.show()

https://github.com/opencv/opencv/tree/master/data/haarcascades

Let's plot the dataset:

In [ ]:
fig, axes = plt.subplots(4, 5, figsize=(16, 16))

ax = axes.ravel()

images = data.lfw_subset()

for i in range(20):
    ax[i].imshow(images[90+i], cmap=plt.cm.gray, interpolation='bicubic') # try with interpolation='bicubic'
    ax[i].axis('off')
    
fig.tight_layout()
plt.show()

### OpenCV

In [ ]:
import io
import zipfile
import requests
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
url = ('https://github.com/ipython-books/'
       'cookbook-2nd-data/blob/master/'
       'family.zip?raw=true')
r = io.BytesIO(requests.get(url).content)
zipfile.ZipFile(r).extractall('data')

In [ ]:
img = cv2.imread('images/person.png')

In [ ]:
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
path = 'data/haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(path)

In [ ]:
for x, y, w, h in face_cascade.detectMultiScale(
        img, 1.3):
    cv2.rectangle(
        img, (x, y), (x + w, y + h), (255, 0, 0), 2)
    
fig, ax = plt.subplots(1, 1, figsize=(8, 8))

rgbimg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

ax.imshow(rgbimg, cmap=plt.cm.gray)
ax.set_axis_off()

### read image

In [ ]:
import cv2 as cv

img = cv.imread("images/person.png", cv.IMREAD_COLOR)
# img = cv.cvtColor(cv.imread("images/person.png"), cv.COLOR_BGR2RGB)

plt.imshow(img)

### resize and blend two images

In [ ]:
!curl https://thispersondoesnotexist.com/image --output images/person2.png

In [ ]:
import cv2

img1 = cv2.imread('images/person.png')
img2 = cv2.imread('images/person2.png')

new_width = 1024
new_height = 1024
new_size = (new_width, new_height)

img1_resized = cv2.resize(img1, new_size, interpolation=cv2.INTER_LINEAR)
img2_resized = cv2.resize(img2, new_size, interpolation=cv2.INTER_LINEAR)

weighted_sum = cv2.addWeighted(img1_resized, 0.5, img2_resized, 0.5,0)
img = cv.cvtColor(weighted_sum, cv.COLOR_BGR2RGB)

plt.imshow(weighted_sum)

### open a webcam stream

In [ ]:
import cv2 as cv
from IPython.display import display, Image
import ipywidgets as widgets
import threading

stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # (FontAwesome names without the `fa-` prefix)
)

def view(button):
    cap = cv2.VideoCapture(0)
    display_handle=display(None, display_id=True)
    i = 0
    while True:
        _, frame = cap.read()
        
        frame = cv2.flip(frame, 1) # if your camera reverses your image
        # frame = cv.Canny(frame, 300, 600)
        
        _, frame = cv2.imencode('.jpeg', frame)
        
        display_handle.update(Image(data=frame.tobytes()))
        
        if stopButton.value==True:
            cap.release()
            display_handle.update(None)
            

display(stopButton)
thread = threading.Thread(target=view, args=(stopButton,))
thread.start()

### haar cascades on webcam stream

In [ ]:
!pip install imutils

In [ ]:
import imutils

detector = cv.CascadeClassifier("data/haarcascade_frontalface_default.xml")

def view(button):
    cap = cv2.VideoCapture(0)
    display_handle=display(None, display_id=True)
    i = 0
    while True:
        _, frame = cap.read()
        
        frame = cv2.flip(frame, 1) # if your camera reverses your image
        # frame = cv.Canny(frame, 300, 600)
        
        frame = imutils.resize(frame, width=720)
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        results = detector.detectMultiScale( gray, scaleFactor=1.05, minNeighbors=5, minSize=(30, 30), flags=cv.CASCADE_SCALE_IMAGE)

        for (fX, fY, fW, fH) in results:
            # extract the face ROI
            faceROI = gray[fY:fY + fH, fX:fX + fW]
            cv.rectangle(frame, (fX, fY), (fX + fW, fY + fH), (0, 255, 0), 2)
        
        _, frame = cv2.imencode('.jpeg', frame)
             
        display_handle.update(Image(data=frame.tobytes()))
        
        if stopButton.value==True:
            cap.release()
            display_handle.update(None)


### Local example to replace face with blur or edges


In [ ]:
import cv2 as cv
import imutils

cap = cv.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()

detector = cv.CascadeClassifier("haarcascades/haarcascade_frontalface_default.xml")


while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    frame = imutils.resize(frame, width=1024)
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    results = detector.detectMultiScale( gray, scaleFactor=1.05, minNeighbors=5, minSize=(30, 30), flags=cv.CASCADE_SCALE_IMAGE)

    for (fX, fY, fW, fH) in results:
		# extract the face ROI
        faceROI = gray[fY:fY + fH, fX:fX + fW]

        print(results)

	    # draw the face bounding box on the frame
        cv.rectangle(frame, (fX, fY), (fX + fW, fY + fH), (0, 255, 0), 2)

        # Canny edge detection.
        edges = cv.Canny(faceROI, 100, 200)

        blur = cv.blur(faceROI, (20, 20), 30)
        print(frame[faceROI])

        # Blur or Edges
        # gray[fY:fY + fH, fX:fX + fW] = blur
        gray[fY:fY + fH, fX:fX + fW] = edges

    # Display the resulting frame
    cv.imshow('frame', gray)

    if cv.waitKey(1) == ord('q'):
        break
# When everything done, release the capture
cap.release()
cv.destroyAllWindows()